In [0]:
from google.colab import drive
import matplotlib as mpl
drive.mount('/content/drive')
kongpath = 'drive/My Drive/kongkat/'
mpl.rcParams.update(mpl.rcParamsDefault)

# Model


## Model class

In [0]:
import torch
import torch.nn as nn
import numpy as np


class VRASAM(nn.Module):
    def __init__(self, z_dim, T, x_dim, h_dim, batch_size=1, x_noise_factor=0.1):
        super(VRASAM, self).__init__()

        self.x_noise_factor = x_noise_factor
        self.z_dim = z_dim
        self.T = T
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.batch_size = batch_size
        self.init_hidden()

        # Everything going on in the network has to be of size:
        # (batch_size, T, n_features)

        # We encode the data onto the latent space using bi-directional LSTM
        self.LSTM_encoder = nn.LSTM(
            input_size=self.x_dim,
            hidden_size=self.h_dim,
            num_layers=1,
            bias=True,
            batch_first=True,
            dropout=0,
            bidirectional=True
        )

        self.fnn_zmu = nn.Linear(
            in_features=2*self.h_dim,
            out_features=self.z_dim,
            bias=True
        )

        self.fnn_zvar = nn.Linear(
            in_features=2*self.h_dim,
            out_features=self.z_dim,
            bias=True
        )

        self.fnn_cmu = nn.Linear(
            in_features=2*self.h_dim,
            out_features=self.z_dim,
            bias=True
        )

        self.fnn_cvar = nn.Linear(
            in_features=2*self.h_dim,
            out_features=self.z_dim,
            bias=True
        )

        # The latent code must be decoded into the original image
        self.LSTM_decoder = nn.LSTM(
            input_size=self.z_dim*2,
            hidden_size=self.h_dim,
            num_layers=1,
            bias=True,
            batch_first=True,
            dropout=0,
            bidirectional=True
        )

        self.fnn_xmu = nn.Linear(
            in_features=2*self.h_dim,
            out_features=self.x_dim,
            bias=True
        )

        self.fnn_xb = nn.Linear(
            in_features=2*self.h_dim,
            out_features=self.x_dim,
            bias=True
        )

    def init_hidden(self):
        self.hidden_state = torch.zeros(2, self.batch_size, self.h_dim)
        self.cell_state = torch.zeros(2, self.batch_size, self.h_dim)
        if torch.cuda.is_available():
            self.hidden_state = self.hidden_state.to(torch.device(0))
            self.cell_state = self.cell_state.to(torch.device(0))

    def encoder(self, x):
        out_encoded, (hidden_T, _) = self.LSTM_encoder(
            x, (self.hidden_state, self.cell_state))

        # Swap batch dimension to batch_first
        hidden_T = hidden_T.transpose(0, 1)

        # Flatten 
        flat_hidden_T = hidden_T.reshape(self.batch_size, 1, 2*self.h_dim)
        
        # Fully connected layer from LSTM to var and mu
        mu_z = self.fnn_zmu(flat_hidden_T)
        sigma_z = nn.functional.softplus(self.fnn_zvar(flat_hidden_T))

        # Calculate the similarity matrix
        S = torch.matmul(
            out_encoded,
            torch.transpose(out_encoded, 1, 2)
        )
        
        S = S / np.sqrt((2 * self.h_dim))

        # Use softmax to get the sum of weights to equal 1
        A = nn.functional.softmax(S, dim=2)
        Cdet = torch.matmul(A, out_encoded)

        # Fully connected layer from LSTM to var and mu
        mu_c = self.fnn_cmu(Cdet)
        sigma_c = nn.functional.softplus(self.fnn_cvar(Cdet))

        return mu_z, sigma_z, mu_c, sigma_c

    def decoder(self, c, z):
        # Concatenate z and c before giving it as input to the decoder
        z_cat = torch.cat(self.T*[z], dim=1)
        zc_concat = torch.cat((z_cat, c), dim=2)
        
        # Run through decoder
        out_decoded, _ = self.LSTM_decoder(zc_concat)

        # Pass the decoder outputs through fnn to get LaPlace parameters
        mu_x = self.fnn_xmu(out_decoded)
        b_x = nn.functional.softplus(self.fnn_xb(out_decoded))

        return mu_x, b_x

    def forward(self, x):
        if self.training:
          var_x = self.x_noise_factor*torch.var(x,dim=1) # (batch_size, x_dim)
          noise = torch.randn(self.T, self.batch_size, self.x_dim)

          # (48, 16, 1) * (16, 1)
          if torch.cuda.is_available():
            noise = noise.to(torch.device(0))

          
          x_noise = torch.mul(noise, torch.sqrt(var_x)) # (T, batch_size, x_dim)
          x = x + torch.transpose(x_noise, 0, 1) # (batch_size, T, x_dim)


        # TODO: Add monte-carlo integration
        outputs = {}
        
        mu_z, sigma_z, mu_c, sigma_c = self.encoder(x)
        
        # Don't propagate gradients through randomness
        with torch.no_grad():
            epsilon_z = torch.randn(self.batch_size, 1, self.z_dim)
            epsilon_c = torch.randn(self.batch_size, self.T, self.z_dim)
        
        if torch.cuda.is_available():
          epsilon_z = epsilon_z.to(torch.device(0))
          epsilon_c = epsilon_c.to(torch.device(0))

        z = mu_z + epsilon_z * sigma_z
        c = mu_c + epsilon_c * sigma_c

        mu_x, b_x = self.decoder(c, z)

        outputs["z"] = z
        outputs["mu_z"] = mu_z
        outputs["sigma_z"] = sigma_z
        outputs["c"] = c
        outputs["mu_c"] = mu_c
        outputs["sigma_c"] = sigma_c
        outputs["mu_x"] = mu_x
        outputs["b_x"] = b_x

        return outputs

    def count_parameters(self):
        n_grad = sum(p.numel() for p in self.parameters() if p.requires_grad)
        n_total = sum(p.numel() for p in self.parameters())
        return n_grad, n_total


## Loss and ReconProb

In [0]:
def ELBO_loss(x, outputs, lambda_KL=0.01, eta_KL=0.01):
    mu_x = outputs['mu_x']
    b_x = outputs['b_x']
    mu_z = outputs["mu_z"]
    sigma_z = outputs["sigma_z"]
    mu_c = outputs["mu_c"]
    sigma_c = outputs["sigma_c"]

    # Initialize Laplace with given parameters.
    pdf_laplace = torch.distributions.laplace.Laplace(mu_x, b_x)
    # Calculate mean of likelihood over T and x-dimension.
    likelihood = pdf_laplace.log_prob(x).mean(dim=1).mean(dim=1)

    # Calculate KL-divergence of p(c) and q(z)
    v_z = sigma_z**2  # Variance of q(z)
    v_c = sigma_c**2  # Variance of p(c)
    kl_z = -0.5 * torch.sum(1 + torch.log(v_z) - mu_z**2 - v_z, dim=2)
    kl_c = -0.5 * torch.sum(1 + torch.log(v_c) - mu_c**2 - v_c, dim=2)

    kl_c = torch.sum(kl_c, dim=1)  # Sum over the T dimension.
    kl_z = kl_z[:, 0]  # Get rid of extra x_dim dimension.

    # Calculate the Evidence Lower Bound (ELBO)
    ELBO = -likelihood + lambda_KL * (kl_z + eta_KL * kl_c)

    # Return mean over all examples in batch
    # =================
    # ====== $ ===== RETURN: SUM instead of MEAN?
    # =================
    return torch.mean(ELBO, dim=0)


def similarity_score(net, x, L):

    with torch.no_grad():
        # Pass sequence through encoder to get params in q(z) and p(c)
        mu_z, sigma_z, mu_c, sigma_c = net.encoder(x)
        
        score = 0

        for _ in range(L):

            # Sample a random c and z vector and reparametrize
            epsilon_c = torch.randn(net.batch_size, net.T, net.z_dim)
            epsilon_z = torch.randn(net.batch_size, 1, net.z_dim)
            
            if torch.cuda.is_available():
                epsilon_c = epsilon_c.to(torch.device(0))
                epsilon_z = epsilon_z.to(torch.device(0))

            c = mu_c + epsilon_c * sigma_c
            z = mu_z + epsilon_z * sigma_z

            # Pass sample through decoder and calculate reconstruction prob
            mu_x, b_x = net.decoder(c, z)
            pdf_laplace = torch.distributions.laplace.Laplace(mu_x, b_x)
            score += pdf_laplace.log_prob(x)

        # Average over number of iterations
        return score/L

def get_sim_scores(net, dataset, L):

    batch_size = min((len(dataset), dataset.T*100))

    loader = DataLoader(dataset, shuffle=False, batch_size=batch_size)

    # Re-initialize network
    net.batch_size = batch_size
    net.init_hidden()
    net.eval()
    all_sim_scores = np.zeros((0, dataset.T))

    for x, label in loader:
        # Modify batch size and hidden state
        if x.shape[0] != batch_size:
            net.batch_size = x.shape[0]
            net.init_hidden()

        # Cast to gpu if available
        if torch.cuda.is_available():
            x = x.to(torch.device(0))
        
        temp_sim = tensor2numpy(similarity_score(net, x, L)[:, -1, 0]).reshape(-1, dataset.T)
        all_sim_scores = np.concatenate((all_sim_scores, temp_sim))

    return all_sim_scores

## Online training and validation functions

In [0]:
from torch.utils.data import DataLoader

def train(dataset, net, optimizer, criterion, batch_size, lamba_kl=0.01, num_workers=0):
    
    # Set network into training mode
    net.batch_size = batch_size
    net.init_hidden()
    net.train()
    
    loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    for x, label in loader:
        
        # Cast to gpu if available
        if torch.cuda.is_available():
            x = x.to(torch.device(0))
        
        # Change batch size in last iteration
        if not x.shape[0] == batch_size:
            net.batch_size = x.shape[0]
            net.init_hidden()

        # Zero gradients
        optimizer.zero_grad()
        # Make forward pass with model
        outputs = net(x)
        # Calculate loss and backprop
        loss = criterion(x, outputs, 0.01, lamba_kl)
        loss.backward()
        optimizer.step()

    # Revert to correct net batch size
    net.batch_size = batch_size
    net.init_hidden()

    return net

def validate(dataset, net, criterion, batch_size, lambda_kl=0.01):
    
    # Set network into evalution mode
    net.batch_size = batch_size
    net.init_hidden()
    net.eval()

    loader = DataLoader(dataset=dataset, batch_size=batch_size)

    total_loss = 0

    for x, label in loader:
        
        # Cast to gpu if available
        if torch.cuda.is_available():
            x = x.to(torch.device(0))

        # Change batch size in last iteration
        if not x.shape[0] == batch_size:
            net.batch_size = x.shape[0]
            net.init_hidden()

        # Make forward pass with model
        outputs = net(x)
        loss = criterion(x, outputs, 0.01, lambda_kl)
        total_loss += loss.item()

    # Set model to train mode and revert batch size
    net.batch_size = batch_size
    net.init_hidden()
    net.train()

    return total_loss/len(loader)

def train_validate(dataset, net, optimizer, criterion, n_epochs, batch_size, train_val_split=(0.8, 0.2), num_workers=0, verbose=True):

    train_size = int(train_val_split[0] * len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_losses = []
    val_losses = []
    lambda_kl = 0
    inc_val = 2/n_epochs

    for e in range(n_epochs):

        # Train model
        net = train(train_set, net, optimizer, criterion, batch_size, lambda_kl, num_workers)

        # Evaluate model on training data again and validate
        with torch.no_grad():
            train_loss = validate(train_set, net, criterion, batch_size, lambda_kl)
            val_loss = validate(val_set, net, criterion, batch_size, lambda_kl)

        if verbose:
            print(
                "Epoch {0}/{1} done!\tTrain loss = {2:.2f}\tVal loss = {3:.2f}".format(
                    e+1, n_epochs, train_loss, val_loss)
            )
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        lambda_kl += inc_val
        
    return net, (train_losses, val_losses)

## Misc function

In [0]:
def get_z_values(net, dataset, batch_size, mu=False):

    z_str = 'mu_z' if mu else 'z'

    # Set batch size to length of dataset    
    net.batch_size = batch_size
    net.init_hidden()
    net.eval()

    loader = DataLoader(dataset=dataset, batch_size=batch_size)

    z_all = np.zeros((0, net.z_dim))
    with torch.no_grad():
        for x, label in loader:
            if torch.cuda.is_available():
                x = x.to(torch.device(0))
            
            # Change batch size in last iteration
            if not x.shape[0] == batch_size:
                net.batch_size = x.shape[0]
                net.init_hidden()
                
            output = net(x)
            
            # Extract z
            z = output[z_str]
            z_all = np.concatenate((z_all, tensor2numpy(z[:,0])))

    # Restore batch_size
    net.batch_size = batch_size
    net.init_hidden()

    return z_all

def tensor2numpy(x):
    if x.requires_grad:
        x = torch.Tensor.cpu(x).detach().numpy()
    else:
        x = torch.Tensor.cpu(x).numpy()
    return x

def save_model(net, name):
    _drive = "/content/drive/My Drive/kongkat/models"
    f_name = name + ".pth"
    f_path = os.path.join(_drive, f_name)
    torch.save(net.state_dict(), f_path)

def load_model(parameters, name):
    _drive = "/content/drive/My Drive/kongkat/models"
    f_name = name + ".pth"
    f_path = os.path.join(_drive, f_name)
    
    model = VRASAM(*parameters)
    model.load_state_dict(torch.load(f_path))
    if torch.cuda.is_available():
        model = model.to(torch.device(0))
    
    return model

# GENDATA

In [0]:
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

class GENDATA_TRAIN(Dataset):

    def __init__(self, N_seq, T_w):

        datafolder = os.path.normpath("drive/My Drive/kongkat/data/GENDATA/")
        path = os.path.normpath(f"{datafolder}/train_data.txt")
        self.data = np.genfromtxt(path)
        self.N_seq = N_seq
        self.T = 96
        self.data = self.data[:(self.N_seq*self.T)] # Extract wanted number of sequences
        self.T_w = T_w # Sliding window length
        self.N_obs = self.data.shape[0] # Total number of observations
        self.N_seq = int(self.N_obs / self.T) # Total number of sequences
        self.N_w = self.N_obs - self.T_w + 1  # Total number of windows
        self.X = torch.Tensor(np.zeros((self.N_w, self.T_w)))
        
        # Define labels for sorted training and test data
        self.time_labels = (np.arange(self.T_w-1, (self.T_w+self.N_w)-1) % self.T) + 1
        
        # Sliding window loop
        for w_start in range(self.N_w):
            # Extract window
            self.X[w_start] = torch.Tensor(self.data[w_start:(w_start+T_w)])

        # Assertions
        assert len(self.time_labels) == len(self.X), "labels and X are not the same size"
        assert N_seq <= int(self.N_obs / self.T), "Tried to load more data than available"

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        x = x.view(*x.shape, 1)
        return x, self.time_labels[idx]

    def get_window(self, idx):
        x = self.X[idx]
        x = x.view(1, -1, 1)
        return x

class GENDATA_VAL(Dataset):

    def __init__(self, T_w):

        datafolder = os.path.normpath("drive/My Drive/kongkat/data/GENDATA/")
        data_path = os.path.normpath(f"{datafolder}/val_data.txt")
        anom_path = os.path.normpath(f"{datafolder}/val_anoms.txt")
        label_path = os.path.normpath(f"{datafolder}/val_labels.txt")

        start_idx = 96 - T_w + 1 # Skip first sequence 
        self.data = np.genfromtxt(data_path)[start_idx:]
        self.anom_labels = np.genfromtxt(anom_path)[1:]
        self.labels = np.genfromtxt(label_path, dtype="|U5")[1:]
        self.T = 96
        self.T_w = T_w # Sliding window length
        self.N_obs = self.data.shape[0] # Total number of observations
        self.N_seq = int(self.N_obs / self.T) # Total number of sequences
        self.N_w = self.N_obs - self.T_w + 1  # Total number of windows
        self.X = torch.Tensor(np.zeros((self.N_w, self.T_w)))
        
        # Define labels for sorted training and val data
        self.time_labels = (np.arange(self.T_w-1, (self.T_w+self.N_w)-1) % self.T) + 1
        
        # Sliding window loop
        for w_start in range(self.N_w):
            # Extract window
            self.X[w_start] = torch.Tensor(self.data[w_start:(w_start+T_w)])

        # Assertions
        assert len(self.time_labels) == len(self.X), "labels and X are not the same size"

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        x = x.view(*x.shape, 1)
        return x, self.time_labels[idx]

    def get_window(self, idx):
        x = self.X[idx]
        x = x.view(1, -1, 1)
        return x

class GENDATA_TEST(Dataset):

    def __init__(self, T_w):

        datafolder = os.path.normpath("drive/My Drive/kongkat/data/GENDATA/")
        data_path = os.path.normpath(f"{datafolder}/test_data.txt")
        anom_path = os.path.normpath(f"{datafolder}/test_anoms.txt")
        label_path = os.path.normpath(f"{datafolder}/test_labels.txt")

        start_idx = 96 - T_w + 1 # Skip first sequence 
        self.data = np.genfromtxt(data_path)[start_idx:]
        self.anom_labels = np.genfromtxt(anom_path)[1:]
        self.labels = np.genfromtxt(label_path, dtype="|U5")[1:]
        self.T = 96
        self.T_w = T_w # Sliding window length
        self.N_obs = self.data.shape[0] # Total number of observations
        self.N_seq = int(self.N_obs / self.T) # Total number of sequences
        self.N_w = self.N_obs - self.T_w + 1  # Total number of windows
        self.X = torch.Tensor(np.zeros((self.N_w, self.T_w)))
        
        # Define labels for sorted training and test data
        self.time_labels = (np.arange(self.T_w-1, (self.T_w+self.N_w)-1) % self.T) + 1
        
        # Sliding window loop
        for w_start in range(self.N_w):
            # Extract window
            self.X[w_start] = torch.Tensor(self.data[w_start:(w_start+T_w)])

        # Assertions
        assert len(self.time_labels) == len(self.X), "labels and X are not the same size"

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        x = x.view(*x.shape, 1)
        return x, self.time_labels[idx]

    def get_window(self, idx):
        x = self.X[idx]
        x = x.view(1, -1, 1)
        return x


# Results plot functions

In [0]:
from matplotlib.lines import Line2D
import torch

def plot_sim(ax, x, samples, sim, min_sim, max_sim, title):

    ax[0].plot(tensor2numpy(x[:, -1, 0]), c='black')
    ax[0].set_title(title)

    # Plot regenerated normal data
    ax[0].plot(samples, 'o', markersize=1, c='darksalmon', alpha=0.3, rasterized=True)
    ax[0].set_xlim((0, 96))
    ax[0].grid(False)
    ax[0].set_xticks([])
    ax[0].set_yticks([])

    # Plot similarity
    ax[1].imshow(tensor2numpy(sim).reshape(1, -1), aspect='auto', cmap='RdYlGn', vmin=min_sim, vmax=max_sim)
    ax[1].grid(False)
    ax[1].set_yticks([])
    ax[1].set_xticks([])

    return ax

def results_plot(ax1, ax2, net, all_sim_scores, L=1, n_samples=10):

    outlier_types = ["Snow", "Fault", "Spike", "Shade", "Cloud"]
    
    # Get min and max similarity score
    min_sim = np.quantile(all_sim_scores, 0.02)
    max_sim = np.quantile(all_sim_scores, 0.9)

    # Load   data
    test_data = GENDATA_TEST(T_w)
    X = test_data.X.reshape(-1, test_data.T, test_data.T_w)
    labels = test_data.labels

    # Extract random normal sequence
    normal_X = X[labels == "None"]
    x_normal = random.choice(normal_X).unsqueeze(2)

    # Allocate outliers array
    X_outliers = torch.Tensor(np.zeros((len(outlier_types), *(x_normal.shape))))

    # Extract specified or random outlier sequence
    for i, out_type in enumerate(outlier_types):
        if out_type == "Shade":
            rand_outlier_idx = 134
        else:
            outlier_idxs = np.arange(X.shape[0])[labels == out_type]
            rand_outlier_idx = random.choice(outlier_idxs)
        X_outliers[i] = X[rand_outlier_idx].unsqueeze(2)

    # Cast to GPU
    if torch.cuda.is_available():
        x_normal = x_normal.to(torch.device(0))
        X_outliers = X_outliers.to(torch.device(0))
    
    # Reshape outliers data
    X_outliers = X_outliers.view(-1, test_data.T_w, 1)

    # Create regeneration
    with torch.no_grad():

        # Set network batch size down and reinitalize hidden- and cell state
        net.batch_size = x_normal.shape[0]
        net.init_hidden()

        # Allocate arrays for normal samples
        normal_samples = np.zeros((len(x_normal), n_samples))

        # Calculate similarity
        sim_normal = similarity_score(net, x_normal, L)[:, -1, 0]

        # Pass sequences through encoder to get params in q(z) and p(c)
        mu_z_normal, sigma_z_normal, mu_c_normal, sigma_c_normal  = net.encoder(x_normal)
        
        # Sample n_samples times
        for i in range(n_samples):
            # Sample a random c and z vector and reparametrize
            epsilon_c_normal = torch.randn(net.batch_size, net.T, net.z_dim)
            epsilon_z_normal = torch.randn(net.batch_size, 1, net.z_dim)

            if torch.cuda.is_available():
                epsilon_c_normal = epsilon_c_normal.to(torch.device(0))
                epsilon_z_normal = epsilon_z_normal.to(torch.device(0))

            c_normal = mu_c_normal + epsilon_c_normal * sigma_c_normal
            z_normal = mu_z_normal + epsilon_z_normal * sigma_z_normal

            # Pass sample through decoder and calculate reconstruction prob
            mu_x_normal, b_x_normal = net.decoder(c_normal, z_normal)
            normal_samples[:, i] = tensor2numpy(mu_x_normal[:, -1, 0])

        # Set network batch size down and reinitalize hidden- and cell state
        net.batch_size = X_outliers.shape[0]
        net.init_hidden()

        # Allocate arrays for outlier samples
        outlier_samples = np.zeros((len(X_outliers),  n_samples))

        sim_outlier = similarity_score(net, X_outliers, L=L)[:, -1, 0]
        mu_z_outlier, sigma_z_outlier, mu_c_outlier, sigma_c_outlier = net.encoder(X_outliers)        

        # Sample n_samples times
        for i in range(n_samples):

            epsilon_c_outlier = torch.randn(net.batch_size, net.T, net.z_dim)
            epsilon_z_outlier = torch.randn(net.batch_size, 1, net.z_dim)

            if torch.cuda.is_available():
                epsilon_c_outlier = epsilon_c_outlier.to(torch.device(0))
                epsilon_z_outlier = epsilon_z_outlier.to(torch.device(0))

            c_outlier = mu_c_outlier + epsilon_c_outlier * sigma_c_outlier
            z_outlier = mu_z_outlier + epsilon_z_outlier * sigma_z_outlier

            mu_x_outlier, b_x_outlier = net.decoder(c_outlier, z_outlier)
            outlier_samples[:, i] = tensor2numpy(mu_x_outlier[:, -1, 0])

    # Reshape outlier similarity and samples
    outlier_samples = outlier_samples.reshape(len(outlier_types), -1, n_samples)
    sim_outlier = sim_outlier.view(len(outlier_types), -1, 1)
    X_outliers = X_outliers.view(len(outlier_types), -1, test_data.T_w, 1)

    # Define range of plotting
    plot_lim = (-0.1, 1.1)

    ## Plot outlier data ##
    outlier_axes = [ax1[:, 1], ax1[:, 2], ax2[:, 0], ax2[:, 1], ax2[:, 2]]
    for i in range(len(outlier_axes)):
        plot_sim(outlier_axes[i], X_outliers[i], outlier_samples[i], sim_outlier[i], min_sim, max_sim, f'{outlier_types[i].capitalize()}')
        outlier_axes[i][0].set_ylim(plot_lim)

    ## Plot normal data ##
    plot_sim(ax1[:, 0], x_normal, normal_samples, sim_normal, min_sim, max_sim, 'Normal data')
    ax1[0, 0].set_ylim(plot_lim)
    ax1[0, 0].set_ylabel('Energy', fontsize=14)
    ax2[0, 0].set_ylabel('Energy', fontsize=14)
    ax1[0, 0].set_yticks([0, 1])
    ax2[0, 0].set_yticks([0, 1])
    ax1[0, 0].yaxis.set_tick_params(labelsize=14)
    ax2[0, 0].yaxis.set_tick_params(labelsize=14)

    return ax1, ax2


# Model training

## Training

In [0]:
import matplotlib.pyplot as plt
import random

# Set seed mu
seed = 7
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

N = 1400  # Number of sequences
T_w = 32  # Window length
batch_size = 5000
train_val_split = (0.8, 0.2)
train_data = GENDATA_TRAIN(N, T_w)

# Model hyperparameters
n_epochs = 50
z_dim = 3
h_dim = 128  # Number of LSTM units in each direction
x_dim = 1
lr = 0.001
criterion = ELBO_loss
parameters = [z_dim, T_w, x_dim, h_dim, batch_size]
net = VRASAM(*parameters)
optimizer = torch.optim.Adam(net.parameters(), lr=lr, amsgrad=True)

# Train model
if torch.cuda.is_available():
    net = net.to(torch.device(0))

pars = {'dataset': train_data, 
        'net': net,
        'optimizer': optimizer,
        'criterion': criterion,
        'n_epochs': n_epochs,
        'batch_size': batch_size,
        'train_val_split': train_val_split,
        'num_workers': 0,
        'verbose': True}

net, (train_losses, val_losses) = train_validate(**pars)


## Vizualization of model

In [0]:
test_data = GENDATA_TEST(T_w)
test_all_sim_scores = get_sim_scores(net, test_data, L=512)

In [0]:
fig1, ax1 = plt.subplots(2, 3, 
                       gridspec_kw={
                           'width_ratios': [1, 1, 1],
                           'height_ratios': [5, 1]},
                       figsize=(10, 2))
fig1.subplots_adjust(hspace=0, wspace=0.05)

fig2, ax2 = plt.subplots(2, 3, 
                       gridspec_kw={
                           'width_ratios': [1, 1, 1],
                           'height_ratios': [5, 1]},
                       figsize=(10, 2))
fig2.subplots_adjust(hspace=0, wspace=0.05)

# Visualize the reconstruction results on normal and outlier series
ax1, ax2 = results_plot(ax1, ax2, net, test_all_sim_scores, L=512, n_samples=50)
fig1.savefig(kongpath+'figures/regen_plot_online1.pdf', format='pdf')
fig2.savefig(kongpath+'figures/regen_plot_online2.pdf', format='pdf')
plt.show()

#fig, ax = plt.subplots(1,1,figsize=(10, 3))
#ax.plot(train_losses, label='Training loss', c='darkslategrey')
#ax.plot(val_losses, label='Validation loss', c='darksalmon')
#ax.set_xlabel('Epochs', fontsize=14)
#ax.set_ylabel('Loss', fontsize=14)
#plt.legend(fontsize=14)
#plt.show()

# Model visualization

## PCA

In [0]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Extract all z_values for training data and standardize
z_all = get_z_values(net, train_data, batch_size=5000)
z_all = StandardScaler().fit_transform(z_all)

pca = PCA()
principal_comps = pca.fit_transform(z_all)
print(pca.explained_variance_ratio_)


In [0]:
fig, ax = plt.subplots(1,1, figsize=(7,6))
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)

scat = ax.scatter(principal_comps[:,0], principal_comps[:,1], s=1, c=train_data.time_labels, cmap='plasma_r', rasterized=True)
ax.set_xticks([])
ax.set_yticks([])
ax.axis('off')
cbar = fig.colorbar(scat, cax=cax, orientation='vertical')
cbar.set_label('Time of day', fontsize=14)
cbar.ax.tick_params(labelsize=14)
ticks = [i*4 for i in range(0, 25, 3)]
ticks[0] += 1
tick_labels = [f"{int(l/4):02}" for l in ticks]
cbar.set_ticks(ticks)
cbar.set_ticklabels(tick_labels)
plt.tight_layout()
plt.savefig('drive/My Drive/kongkat/figures/beautiful_online_pca.pdf', format='pdf')
plt.show()

# Outlier visualization

In [0]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Process test data
test_labels = test_data.labels
test_anoms = test_data.anom_labels
test_z_all = get_z_values(net, test_data, batch_size=5000, mu=True)

# Extract all z_values for training data and standardize
test_z_all = StandardScaler().fit_transform(test_z_all)

pca = PCA()
principal_comps = pca.fit_transform(test_z_all)
principal_comps = principal_comps.reshape(len(test_labels), 96, z_dim)
print(pca.explained_variance_ratio_)

In [0]:
n_normals = 5
normal_idxs = np.arange(test_labels.shape[0])[test_labels == "None"]
plot_idxs = list(np.random.choice(normal_idxs, size=n_normals, replace=False))
outlier_types = ["Snow", "Fault", "Spike", "Shade", "Cloud"]

## Plot random outliers (comment for specific outliers below)
#for out in outlier_types:
#    outlier_idxs = np.arange(test_labels.shape[0])[test_labels == out]
#    plot_idxs.append(np.random.choice(outlier_idxs))

## Uncomment and insert chosen indices into dictionary once found ##
chosen_ones = {'Snow': 1106,
               'Fault': 546,
               'Spike': 866,
               'Shade': 568,
               'Cloud': 436}
plot_idxs.extend(list(chosen_ones.values()))

exp_var = sum(pca.explained_variance_ratio_[:2])*100
col2label = {"None": 'black',
             "Snow": 'darksalmon', 
             "Fault": 'springgreen', 
             "Spike": 'skyblue', 
             "Shade": 'fuchsia', 
             "Cloud": 'orangered'}


In [0]:
from matplotlib.lines import Line2D

fig, ax = plt.subplots(1,1, figsize=(7,6))

for idx in plot_idxs:
    lab = test_labels[idx]
    if lab == "None":
        comp = principal_comps[idx]
        ax.plot(comp[:,0].T, comp[:,1].T, 'o-', c=col2label[lab], markersize=5, linewidth=1)
    else:
        comp = principal_comps[idx]
        anom_start, anom_end = test_anoms[idx]
        all_idxs = np.arange(96)
        out_idxs = (all_idxs<=anom_end)*(all_idxs>=anom_start)
        norm_idxs1 = all_idxs<=anom_start
        norm_idxs2 = all_idxs>=anom_end
        ax.plot(comp[norm_idxs1,0].T, comp[norm_idxs1,1].T, 'o-', c=col2label["None"], markersize=5, linewidth=1)
        ax.plot(comp[norm_idxs2,0].T, comp[norm_idxs2,1].T, 'o-', c=col2label["None"], markersize=5, linewidth=1)
        ax.plot(comp[out_idxs,0].T, comp[out_idxs,1].T, 'o-', c=col2label[lab], markersize=5, linewidth=1)

lines = [Line2D([0], [0], color=c, linewidth=3, linestyle='-', marker='o') for k,c in col2label.items()]
leg_labels = [name if name!="None" else "Normal" for name in col2label.keys()]
#leg_labels = ["Normal"] + [name+f" (idx {plot_idxs[n_normals+i]})" for i,name in enumerate(list(col2label.keys())[1:])]
#ax.set_xlabel('Principal component 1')
#ax.set_ylabel('Principal component 2')
ax.tick_params(axis='both', which='major')
ax.set_xticks([])
ax.set_yticks([])
ax.axis('off')
plt.legend(lines, leg_labels, frameon=False, fontsize=14, loc=(-0.1,0))
#plt.title(f'Normal- and outlier sequences in z-space ({exp_var:.2f}% explained variance)', fontsize=20)
plt.tight_layout()
plt.savefig('drive/My Drive/kongkat/figures/online_outlier_pca.pdf', format='pdf')
plt.show()

# Classification

In [0]:
from sklearn.metrics import roc_curve, roc_auc_score

def plotRocCurve(ax, ytrue, sim_scores):
    #Extract propriate values for roc-curve
    ytrue_roc = []
    for i in range(len(ytrue)):
      if ytrue[i] == 0:
        ytrue_roc.append(1)
      else:
        ytrue_roc.append(0)

    fpr, tpr, thresholds = roc_curve(ytrue_roc, sim_scores)
    auc = roc_auc_score(ytrue_roc, sim_scores)
    random = [0,1]

    # plot the roc curve for the model
    ax.plot(fpr, tpr, linestyle='--', label=f'Threshold prediction (AUC={auc:.2f})', c='darkslategrey')
    ax.plot(random, random, linestyle='--', label='No information', c='darksalmon')
    # axis labels
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.tick_params(axis='both', which='major')
    ax.set_xticks([0, 0.5, 1])
    ax.set_yticks([0, 0.5, 1])
    # show the legend
    ax.legend()

    return thresholds, auc

In [0]:
test_sim_scores = test_all_sim_scores.sum(axis=1)
test_ytrue = test_labels != "None"
fig, ax = plt.subplots(1, 1, figsize=(5,3))
thresholds, auc = plotRocCurve(ax, test_ytrue, test_sim_scores)
plt.savefig(kongpath+'figures/ROC_online.pdf', format='pdf')
plt.show()

In [0]:
# Load threshold data
val_data = GENDATA_VAL(T_w)
val_all_sim_scores = get_sim_scores(net, val_data, L=512)
val_sim_scores = val_all_sim_scores.sum(axis=1)
val_ytrue = val_data.labels != 'None'

In [0]:
fig, ax = plt.subplots(1, 1, figsize=(15,5))
acc = np.zeros(len(thresholds))

for i,thres in enumerate(thresholds):
    val_ypred = val_sim_scores < thres
    trues = sum(val_ypred == val_ytrue)
    total = len(val_ypred)
    acc[i] = trues/total

ax.bar(range(len(thresholds)), acc, color='darkslategrey')
ax.set_xticks([])
ax.set_xlabel('Reconstruction likelihood thresholds', fontsize=16)
ax.set_ylabel('Accuracy', fontsize=16)
for index, value in enumerate(acc):
    plt.text(index, value-0.05, f'{value*100:.0f}', ha='center', color='white', fontsize=10)
plt.show()

# Extract best threshold
threshold = thresholds[np.argmax(acc)]

In [0]:
# Process test data
test_sim_scores = test_all_sim_scores.sum(axis=1)
test_labels = test_data.labels
test_ytrue = test_labels != 'None'

In [0]:
from collections import Counter
import pandas as pd
import seaborn as sns
import matplotlib as mpl

class_names = ["None", "Snow", "Fault", "Spike", "Shade", "Cloud"]

test_ypred = test_sim_scores < threshold
correct = test_ytrue == test_ypred
conf_mat = np.zeros((2, len(class_names)))

total_dict = Counter(test_labels)

for i,cls in enumerate(class_names):
    acc_cls = sum(correct[test_labels == cls])/total_dict[cls]
    if cls == "None":
        conf_mat[0,i] = acc_cls
        conf_mat[1,i] = 1 - acc_cls
    else:
        conf_mat[0,i] = 1 - acc_cls
        conf_mat[1,i] = acc_cls

class_names = ["Normal", "Snow", "Fault", "Spike", "Shade", "Cloud"]
fig, ax = plt.subplots(1, 1, figsize=(6,2))
df_cm = pd.DataFrame(conf_mat, ['Normal', 'Outlier'], class_names)
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, fmt='.2f', annot=True, annot_kws={"size": 10}, ax=ax, cmap='Greys', cbar=False)# font size

ax.set_yticklabels(['Normal', 'Outlier'], va='center')
ax.set_xlabel("True value", fontsize=12)
ax.set_ylabel("Prediction", fontsize=12)
plt.tight_layout()
plt.savefig(kongpath+'figures/confusion_matrix_online.pdf', format='pdf')
plt.show()
mpl.rcParams.update(mpl.rcParamsDefault)
print(f'Final accuracy: {sum(correct)/len(correct):.2f}')